In [25]:
import pandas as pd
import re
import zipfile
import numpy as np
from tqdm import tqdm

In [2]:
zipname = "vectors.zip"
filename = "gensim_glove_vectors.txt"

with zipfile.ZipFile(zipname, 'r') as zip_ref:
    zip_ref.extractall('.')

BadZipFile: File is not a zip file

In [ ]:
# užtruks 331205 iteracijas, <1 min.
embeddings = {}
with open(filename, 'r') as f:
  for i, line in tqdm(enumerate(f)):
    if i == 0:
      continue
    values = line.split(' ')
    word = values[0]
    vector = np.array(values[1:], dtype='float32')
    embeddings[word] = vector

len(embeddings)

In [84]:
df = pd.read_excel('data.xlsx')
df = df.dropna(subset=['ingredients'])
df.head()

,title,authors_desc,portions,prep_time,ingredients,steps
0,"Nuostabūs sviestiniai sausainiai ""Citrinėlės""","Kuomet pabosta arbata su citrina, galime pasim...",Porcijų skaičius: 15,Paruošimo laikas: Apie 1 val.,1.5 stiklinės miltų; 0.5 stiklinės cukraus; 15...,"Dubenyje išplakame kiaušinį, išspaudžiame 1 ci..."
1,Greiti kokosiniai - sviestiniai sausainiai,"Iškepa nuostabūs, trapūs, kvapnūs sausainėliai.",Porcijų skaičius: 30,Paruošimo laikas: Apie 30 min.,125 gramai sviesto; 125 gramai kvietinių miltų...,"Į dubenį dėti sviestą, berti cukrų, vanilinį c..."
2,Benedikto kiaušiniai su ZIGMO lašiša,Benedikto kiaušiniai geriausias pasirinkimas K...,Porcijų skaičius: 4,Paruošimo laikas: Apie 10 min.,100 gramų ZIGMO sūdytos pjaustytos lašišos fil...,Į verdantį pasūdytą vandenį vieną po kito įmuš...
3,Citrininis maskarponinis tinginių Napoleonas i...,"Panašu, kad atradau antrą savo mėgstamiausią t...",Porcijų skaičius: 8,Paruošimo laikas: Apie 1 val.,500 gramų bemielės sluoksniuotos tešlos ...,Sluoksniuotos tešlos lakštus atitirpinau ir pa...
5,BBQ šonkauliai,"Tai skanus ir išskirtinis patiekalas, kuris le...",Porcijų skaičius: 4,Paruošimo laikas: Daugiau nei 1 val.,1 kilogramas kiaulienos šonkaulių; 1 vienetas ...,Visus prieskonius ir kavą sumaišyti inde. Šonk...


In [5]:
len(df)

17029

In [98]:
df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\s+', ' ', x))

for col in df.columns:
    df[col] = df[col].str.lower()

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\b(gr\.?|gram\w*)\b', 'gramai', x))
df['steps'] = df['steps'].apply(lambda x: re.sub(r'\b(gr\.?|gram\w*)\b', 'gramai', x))

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\b(kg\.?|kilo\w*)\b', 'kilogramai', x))
df['steps'] = df['steps'].apply(lambda x: re.sub(r'\b(kg\.?|kilo\w*)\b', 'kilogramai', x))

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\b(kg\.?|kilo\w*)\b', 'kilogramai', x))
df['steps'] = df['steps'].apply(lambda x: re.sub(r'\b(kg\.?|kilo\w*)\b', 'kilogramai', x))

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\b(ml\.?|mili\w*)\b', 'mililitrai', x))
df['steps'] = df['steps'].apply(lambda x: re.sub(r'\b(ml\.?|mili\w*)\b', 'mililitrai', x))

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\blit\w*\b', 'litrai', x))
df['steps'] = df['steps'].apply(lambda x: re.sub(r'\blit\w*\b', 'litrai', x))

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\b(cm\.?|centi\w*)\b', 'centimetrai', x))
df['steps'] = df['steps'].apply(lambda x: re.sub(r'\b(cm\.?|centi\w*)\b', 'centimetrai', x))

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\bstikl\w*\b', 'stiklinės', x))
df['steps'] = df['steps'].apply(lambda x: re.sub(r'\bstikl\w*\b', 'stiklinės', x))

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\bv\.š\.\b', 'virtuviniai šaukštai', re.sub(r'\ba\.š\.\b', 'arbatiniai šaukšteliai', x)))
df['steps'] = df['steps'].apply(lambda x: re.sub(r'\bv\.š\.\b', 'virtuviniai šaukštai', re.sub(r'\ba\.š\.\b', 'arbatiniai šaukšteliai', x)))

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'<.*?>', '', x))
df['steps'] = df['steps'].apply(lambda x: re.sub(r'<.*?>', '', x))

df['ingredients'] = df['ingredients'].apply(lambda x: re.sub(r'\(.*?\)', '', x))
df['ingredients'] = df['ingredients'].apply(lambda x: x.replace(';', ','))

In [89]:
unique_symbols = set()
for text in df['ingredients']:
    for symbol in text:
        unique_symbols.add(symbol)
print(unique_symbols)

{'>', 'ū', '̨', ':', 'ų', 'ž', 'x', '(', 'a', '5', '*', '_', '’', '‚', 'n', '4', 'k', 'z', 'b', 'r', '.', 'c', 'u', 'ą', '/', 'š', 'p', 's', 'ę', '“', 't', 'y', '̇', 'è', 'e', ';', 'o', '-', '~', '8', 'ñ', 'm', 'é', '6', '̄', '0', '!', 'g', '%', '&', 'l', '9', ')', 'ė', '1', 'd', '3', '̌', '–', 'v', 'j', '=', '”', 'į', 'č', '`', 'h', 'w', 'f', ' ', 'q', '7', '+', 'i', "'", '2', '¼', '"', '„', '‘', ','}


In [110]:
df['input'] = (
    'pavadinimas: ' + df['title'] + '; ' +
    'ingridientai: ' + df['ingredients'] + ';'
    'aprašymas: ' + df['authors_desc'] + '; ' +
    'porcijų skaičius: ' + df['portions'].astype(str) + '; ' +
    df['prep_time']
)
df['output'] = df['steps']

alphabet = 'abcdefghijklmnopqrstuvwxyząčęėįšųūž.,!?":;~0123456789%+-= '
pattern = f'[^{re.escape(alphabet)}]'
df['input'] = df['input'].apply(lambda x: re.sub(pattern, '', str(x).lower()))
df['output'] = df['output'].apply(lambda x: re.sub(pattern, '', str(x).lower()))
df.head()

,title,authors_desc,portions,prep_time,ingredients,steps,input,output
0,"nuostabūs sviestiniai sausainiai ""citrinėlės""","kuomet pabosta arbata su citrina, galime pasim...",porcijų skaičius: 15,paruošimo laikas: apie 1 val.,"1.5 stiklinės miltų, 0.5 stiklinės cukraus, 15...","dubenyje išplakame kiaušinį, išspaudžiame 1 ci...",pavadinimas: nuostabūs sviestiniai sausainiai ...,"dubenyje išplakame kiaušinį, išspaudžiame 1 ci..."
1,greiti kokosiniai - sviestiniai sausainiai,"iškepa nuostabūs, trapūs, kvapnūs sausainėliai.",porcijų skaičius: 30,paruošimo laikas: apie 30 min.,"125 gramai sviesto, 125 gramai kvietinių miltų...","į dubenį dėti sviestą, berti cukrų, vanilinį c...",pavadinimas: greiti kokosiniai - sviestiniai s...,"į dubenį dėti sviestą, berti cukrų, vanilinį c..."
2,benedikto kiaušiniai su zigmo lašiša,benedikto kiaušiniai geriausias pasirinkimas k...,porcijų skaičius: 4,paruošimo laikas: apie 10 min.,100 gramai zigmo sūdytos pjaustytos lašišos fi...,į verdantį pasūdytą vandenį vieną po kito įmuš...,pavadinimas: benedikto kiaušiniai su zigmo laš...,į verdantį pasūdytą vandenį vieną po kito įmuš...
3,citrininis maskarponinis tinginių napoleonas i...,"panašu, kad atradau antrą savo mėgstamiausią t...",porcijų skaičius: 8,paruošimo laikas: apie 1 val.,"500 gramai bemielės sluoksniuotos tešlos , 400...",sluoksniuotos tešlos lakštus atitirpinau ir pa...,pavadinimas: citrininis maskarponinis tinginių...,sluoksniuotos tešlos lakštus atitirpinau ir pa...
5,bbq šonkauliai,"tai skanus ir išskirtinis patiekalas, kuris le...",porcijų skaičius: 4,paruošimo laikas: daugiau nei 1 val.,"1 kilogramai kiaulienos šonkaulių, 1 vienetas ...",visus prieskonius ir kavą sumaišyti inde. šonk...,pavadinimas: bbq šonkauliai; ingridientai: 1 k...,visus prieskonius ir kavą sumaišyti inde. šonk...
